In [1]:
import os

In [2]:
%pwd

'c:\\Users\\dhira\\Desktop\\MLflow\\MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\dhira\\Desktop\\MLflow\\MLflow-DVC'

In [9]:
#Training entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int


In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
# 5. Update the configuration manager in src config

class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
             root_dir = Path(config.root_dir),
             base_model_path = Path(config.base_model_path),
             updated_base_model_path = Path(config.updated_base_model_path),
             params_image_size = self.params.IMAGE_SIZE,
             params_learning_rate = self.params.LEARNING_RATE,
             params_include_top = self.params.INCLUDE_TOP,
             params_weights = self.params.WEIGHTS,
             params_classes = self.params.CLASSES

        )

        return prepare_base_model_config

In [13]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [14]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )

        self.save_model(path = self.config.base_model_path, model = self.model)
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation = "softmax"
        ) (flatten_in)

        full_model = tf.keras.model.Model(
             inputs = model.input,
             outputs = prediction
        )
        
        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"]
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate= self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model = self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [15]:
# Torch code

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from pathlib import Path


class PrepareBaseModel:
    def __init__(self, config):
        self.config = config
        self.model = None
        self.full_model = None

    def get_base_model(self):
        # Load pretrained VGG16
        self.model = models.vgg16(
            weights=models.VGG16_Weights.DEFAULT if self.config.params_weights == "imagenet" else None
        )

        # Remove classifier if include_top = False
        if not self.config.params_include_top:
            self.model.classifier = nn.Identity()

        # Save the base model
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        # Freeze layers
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for param in list(model.parameters())[:-freeze_till]:
                param.requires_grad = False

        # Replace classifier head
        in_features = model.classifier[-1].in_features if isinstance(model.classifier, nn.Sequential) else 25088
        model.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, classes),
            nn.Softmax(dim=1)
        )

        # Define loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)

        print(model)
        return model, criterion, optimizer

    def update_base_model(self):
        self.full_model, self.criterion, self.optimizer = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: nn.Module):
        torch.save(model.state_dict(), path)






In [18]:
try:
   config = ConfigurationManager()
   prepare_base_model_config = config.get_prepare_base_model_config()
   prepare_base_model = PrepareBaseModel(config = prepare_base_model_config)
   prepare_base_model.get_base_model()
   prepare_base_model.update_base_model()

except Exception as e:
   raise e


[2025-09-07 18:35:15,609: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-09-07 18:35:15,611: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-09-07 18:35:15,612: INFO: common: created directory at: artifacts]
[2025-09-07 18:35:15,613: INFO: common: created directory at: artifacts/prepare_base_model]
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\dhira/.cache\torch\hub\checkpoints\vgg16-397923af.pth


100%|██████████| 528M/528M [00:15<00:00, 35.9MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1